In [16]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

In [27]:
dataset = pd.read_csv('dataset.csv', encoding = "latin1", names=["Sentences", "Intent"])
print(dataset)

                                              Sentences          Intent
0                                      Need help pleese  commonQ.assist
1                                             Need help  commonQ.assist
2                                      I need some info  commonQ.assist
3                                     Will you help me?  commonQ.assist
4                                 What else can you do?  commonQ.assist
...                                                 ...             ...
1108  borrowed amount can be used for which of the p...  faq.borrow_use
1109  borrowed amount given can used by me for what ...  faq.borrow_use
1110  borrowed amount given can used by me for which...  faq.borrow_use
1111  borrowed money can be used for which of the pu...  faq.borrow_use
1112  borrow money given can used by me for what rea...  faq.borrow_use

[1113 rows x 2 columns]


In [28]:
unique_intents = set(list(dataset['Intent']))
print(unique_intents, len(unique_intents))

{'faq.approval_time', 'faq.biz_simpler', 'faq.biz_new', 'commonQ.how', 'faq.application_process', 'faq.borrow_limit', 'faq.borrow_use', 'commonQ.query', 'faq.banking_option_missing', 'faq.apply_register', 'commonQ.bot', 'commonQ.name', 'faq.bad_service', 'faq.biz_category_missing', 'commonQ.wait', 'commonQ.just_details', 'commonQ.not_giving', 'faq.aadhaar_missing', 'contact.contact', 'commonQ.assist', 'faq.address_proof'} 21


In [34]:
sentences = list(dataset['Sentences'])
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']
